# Intro to Socket Programming
## Overview

### What You'll Learn
In this section, you'll learn
1. What sockets are
1. How to use them to send and receive messages between computers

### Prerequisites
Before starting this section, you should have an understanding of
1. [Basic Python](https://github.com/HackBinghamton/PythonWorkshop)

### Introduction
"Sockets" are objects that allow your programs to send and receive messages on the Internet. We'll learn what they are and the basics of how to use them.

## What Are Sockets?

"Sockets" give your program access to the Internet -- there's no way to connect to the Internet without using sockets in some way or another. Thanks to some excellent design in UNIX-based operating systems, we can use them almost exactly the same as the way we use files, with functions analagous to `open()`, `close()`, `read()`, and `write()`.

## How Do Sockets Work?

Let's say we have two computers, A and B, that want to talk or otherwise exchange information (e.g. files, video streams, etc.). We'll have to pick one computer to act as a "server" (i.e. the computer that sits and waits for a connection from the other), and we'll have to choose one to act as the "client" (i.e. the computer that initiates a connection to the server). Let's say A is the client and B is the server.

Now, we can go ahead and write client-side and server-side code for A and B, respectively (we'll get into the details of that below).

Once the code is written, we can run the server code on B, which will open up a socket for incoming connections. B will sit and wait as long as we let it, until we run A's code and have it connect to B's open socket. Once they connect, A and B can send and receive messages from one another, and eventually they'll close their connection.

This is just a generalization of how networked programs work, for reference. In the next subsection, we'll fill in the gaps on how this actually happens.

*Side Note:* One program always has to act as a server -- that is, you can't have two client-side programs attempt to connect to one another. If you want to try something that works peer-to-peer, you could have each computer run both a client-side and server-side program, so that they can connect to one another freely.


## How Do I Use Sockets?

### Importing the `socket` Library and Creating a Socket

To import Python's `socket` library, we can simply use:

```python
import socket
```

To actually create a socket, we can use:

```python
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
```

This will give us a new socket that uses TCP and IPv4. **For most purposes, this is what you'll end up using.** No worries if those acronyms sound like gibberish, since we'll be covering those in the [TCP/IP Stack]() and the [IP Addresses and DNS]() sections, respectively.

### General `socket.socket` Reference

To use sockets, you'll just need to understand the following functions:

**For server-side programs:**

* `.bind((ip, port))` - Attach this socket to an IP address and port combination
* `.listen(num_connections)` - Put the socket into "listening" mode, where it can accept incoming connections
* `.accept()` - Accept an incoming connection and get a new socket object for it

**For client-side programs:**

* `.connect((ip, port))` - For connecting to a server

**For both:**

* `.send(bytes)` - For sending bytes to the other end of a connection
* `.recv(num_bytes)` - For receiving `num_bytes` bytes from the other end of a connection
* `.close()` - For closing a socket

We'll be going over these in more depth in the next couple of subsections!

### Building a Client-side Socket Program

Creating a client-side socket is relatively simple, as there are four main steps:

1. Construct the socket
2. Connect the socket to the server
3. Send/receive data
4. Close the socket

Each of these steps can be found in the following code (which you can feel free to repurpose -- it's pretty standard):

```python
# 1.a Import the socket library
import socket

# 1.b Create the socket object
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 2.a Specify what server we'd like to connect to
server_ip = "192.168.0.36"
server_port = 9090
server_address = (server_ip, server_port)

# 2.b Connect to the server
sock.connect(server_address)

# 3. Send and receive data
# ...
# ... (Covered in two subsections!)
# ...

# 4. Close the socket
sock.close()
```

The only method call in here unique to the client-side code is `.connect()`, and it's worth discussing what parameters it takes.

An **IP address** or just **IP** is a number that uniquely identifies a given computer on a network -- it's just like your postal address identifies your house. If someone knows it, then they can send you messages.

A **port number** specifies *which socket* to connect to on a computer. Port numbers are necessary since any computer could run up to 65,535 networked programs, and so we need to know which one we're connecting to.

For example, the SSH protocol (which you might have used to connect to `harvey` or `remote` for class) runs on port 22, while HTTP runs on port 80, and HTTPS runs on port 443!

These port numbers are all standardized, so as long as we know the IP address of the computer we're connecting to and the port number of the protocol we're using, then we can connect without a problem.

We'll discuss how we get from hostnames like `cs.binghamton.edu` or `www.google.com` to IP addresses in our [IP Addresses and DNS]() section!


### Building a Server-side Socket Program

When creating a server-side program, there are a few more steps to keep track of:

1. Construct the socket (same as client-side)
2. "Bind" the socket to our IP address
3. Set the socket to listen for new connections
4. Accept connections as they come in
5. For each connection, send and receive data (same as client-side)
6. Close the accepted client socket
6. Close the server socket

In code, these steps might line up like so:

```python
# 1.a Import the socket library
import socket

# 1.b Create the server socket
server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 2.a Specify where we'd like to host
#    This is a little weird -- check the notes below!
address = ("0.0.0.0", 9090)

# 2.b Bind our socket to our address
server_sock.bind(address)

# 3. Start listening for incoming connections
server_sock.listen(1)

# 4. Accept an incoming connection
#    Note the creation of the new "client_sock"!
client_sock, client_address = server_sock.accept()

# 5. Send and receive data
# ...
# ... (Covered in the next subsection, but make sure to use client_sock!)
# ...

# 6. Close the connected client socket
client_sock.close()

# 7. Close the server socket
server_sock.close()
```

You might notice that some things are a little bit funky -- why do we have separate client and server sockets? Why do we use `"0.0.0.0"` as our IP address?

To answer the first question, our server-side socket behaves slightly differently than a client-side socket. **When we accept a connection from `server_sock`, we are provided a brand new `socket` object that is directly connected to our client.** The reason is that it allows our server-side socket to continue accepting other clients (in a multi-threaded server) while we work with the client-specific socket.

To answer the second question, there are a couple of special IP addresses to keep in mind when hosting:

* `"0.0.0.0"` tells your computer, "Make this socket available for everyone on my network to connect to!"
* `"127.0.0.1"` or `"localhost"` tells your computer, "Make this socket private so only other programs on my computer can see it."

Each of these have their uses, but keep in mind that `"0.0.0.0"` will allow other people on your network to connect to your computer, which you may or may not want.

*Question:* This program only accepts one client, talks with it, and then exits -- how might we be able to keep accepting new clients, or multiple clients at a time?

*Answer:* <spoiler>We can use a `while` loop to continually accept new clients, and/or use threading to handle multiple clients at once! (No need to know that for this workshop though, but it might come in handy someday!)</spoiler>

### Sending and Receiving Data with Sockets

Now that we know how to create and client-side and server-side sockets, let's look at how we can exchange data with them. There are two main functions, `.send()` and `.recv()`, that let us do this.

#### Client-side Perspective

Let's look at a client-side view of sending a simple hello message to a server:

```python
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0)

sock.connect(("127.0.0.1", 7070))

sock.send(b"Hello! Is this the server?")

response_bytes = sock.recv(512)

print(response_bytes)

sock.close()
```

This code creates a socket, connects to the server, and then sends a byte-string into the socket, effectively sending that message to the server. `.send()` takes any object of type `bytes`, and adding a `b` before your first quotation mark in a string will make it a byte-string, which works great for this purpose.

Afterwards, we use `.recv()` to receive a response from the server. `.recv()` takes a number representing the largest amount of data you'd like to receive. Usually, it's a good idea to leave it at some large multiple of 2, like 512, 1024, or 2048. Once the message is received, it returns it as a `bytes` object.

*Note:* You can turn `bytes` objects into strings by using the `.decode()` function on them! (i.e. `response_bytes.decode()`)

#### Server-side Perspective

The server's view would probably look something like this:

```python
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0)

sock.bind(("127.0.0.1", 7070))

sock.listen(5)

connection, address = sock.accept()

client_hello = connection.recv(512)

print(client_hello)

connection.send(b"Yep! This is the server.")

connection.close()

sock.close()
```

We create a socket, bind to the private `localhost` address on port 7070, listen for connections, and then accept one. Once we accept the connection, we have to receive the client's hello message with `.recv()`, and then send our response with `.send()` -- and these functions work just the same as when we used them on the client side.

When writing socket programs, keep in mind what order the client and server exchange messages, and make sure to match that on both the client and server ends.

#### Exercise

If you have Python installed on your computer, or have access to `remote` or `harvey`, try these programs out! Some food for thought: Which program should you run first?

The next sections will be interactive with code blocks -- we just couldn't do this one in code blocks since you can only run one block at a time!

*Note*: If using SSH to `remote` or `harvey`, you'll need two SSH windows open, since you'll need to run two programs at once (i.e. the client and server).

## Challenge

Now that you understand everything you need to know to write socket programs, write a simple chat program where you taken user input, and the server and client can chat with one another as long as they'd like!

**Security Notice:** These sockets as we've been using them *offer no security* on their own -- anyone on the same Wi-Fi network as the client or server can intercept and read messages! If you're interested in learning to make sockets secure, you should definitely pick up some cryptography skills at our next workshop and check out the [Python SSL documentation](https://docs.python.org/3/library/ssl.html)!

# Next Section: [The TCP/IP Stack and Transport-Layer Protocols]()